In [3]:
import os
import shutil
# 소스 디렉토리 및 대상 디렉토리 경로 설정
source_root_label = '/home/min/dev_ws/machine_learning/project_3rd/data/traffic_detection/VL'
source_root_image = '/home/min/dev_ws/machine_learning/project_3rd/data/traffic_detection/VS'
target_dir_label = '/home/min/dev_ws/machine_learning/project_3rd/data/traffic_detection/label'
target_dir_image = '/home/min/dev_ws/machine_learning/project_3rd/data/traffic_detection/image'

# 대상 디렉토리 생성
os.makedirs(target_dir_label, exist_ok=True)
os.makedirs(target_dir_image, exist_ok=True)

# 모든 하위 폴더 탐색
for root_label, dirs_label, files_label in os.walk(source_root_label):
    for file_label in files_label:
        # JSON 파일일 경우만 복사
        if file_label.endswith('.json'):
            src_path_label = os.path.join(root_label, file_label)
            dst_path_label = os.path.join(target_dir_label, file_label)
            # 파일 복사
            shutil.copy(src_path_label, dst_path_label)

for root_image, dirs_image, files_image in os.walk(source_root_image):
    for file_image in files_image:
        # Jpg 파일일 경우만 복사
        if file_image.endswith('.jpg'):
            src_path_image = os.path.join(root_image, file_image)
            dst_path_image = os.path.join(target_dir_image, file_image)
            # 파일 복사
            shutil.copy(src_path_image, dst_path_image)


In [4]:
import os
import json
import cv2
# JSON 파일이 있는 디렉토리 경로 설정
label_dir = '/home/min/dev_ws/machine_learning/project_3rd/data/traffic_detection/label'

# 클래스명을 저장할 집합(set) 생성
class_names = set()

# 모든 JSON 파일 탐색
for filename in os.listdir(label_dir):
    if filename.endswith('.json'):
        file_path = os.path.join(label_dir, filename)
        
        # JSON 파일 열기
        with open(file_path, 'r') as f:
            data = json.load(f)
            
            # JSON 데이터에서 클래스명 추출하여 집합에 추가
            objects = data.get('objects', [])
            for obj in objects:
                class_name = obj.get('class_name', None)
                if class_name is not None:
                    class_names.add(class_name)

# 결과 출력
total_classes = len(class_names)
print("클래스 총 개수:", total_classes)
print("클래스명 목록:")
for class_name in sorted(class_names):
    print(class_name)

클래스 총 개수: 3
클래스명 목록:
invisible_signal
pedestrian_signal
vehicular_signal


바운딩 박스 좌상단 / 우하단 좌표
-> 꼭짓점 네개로 바꾸기~~~~~~~~~~~!!!

In [20]:
import os
import json

label_dir = '/home/min/dev_ws/machine_learning/project_3rd/data/traffic_detection/label'  # JSON 파일이 위치한 디렉토리 경로

# 모든 JSON 파일 탐색
for filename in os.listdir(label_dir):
    if filename.endswith('.json'):
        file_path = os.path.join(label_dir, filename)
        
        # JSON 파일 열기
        with open(file_path, 'r') as f:
            data = json.load(f)
            
            # JSON 데이터에서 객체 추출하여 바운딩 박스 좌표 변환
            objects = data.get('objects', [])
            for obj in objects:
                points = obj.get('data', [])
                if len(points) == 2:
                    pt1 = (int(points[0][0]), int(points[0][1]))
                    pt2 = (int(points[1][0]), int(points[1][1]))
                    
                    # 네 개의 꼭짓점 좌표
                    top_left = pt1
                    bottom_right = pt2
                    top_right = (pt2[0], pt1[1])
                    bottom_left = (pt1[0], pt2[1])
                    
                    # 네 개의 꼭짓점 좌표를 객체의 'data'에 다시 저장
                    obj['data'] = [top_left, top_right, bottom_right, bottom_left]

        # 변경된 데이터를 다시 JSON 파일로 저장
        with open(file_path, 'w') as f:
            json.dump(data, f, indent=4)


In [1]:
import os
import json
import cv2

# JSON 파일이 있는 디렉토리 경로 설정
label_dir = '/home/min/dev_ws/machine_learning/project_3rd/data/traffic_detection/label'
image_dir = '/home/min/dev_ws/machine_learning/project_3rd/data/traffic_detection/image'
# 신호등 색깔을 저장할 집합(set) 생성
signal_colors = set()

# 모든 JSON 파일 탐색
for filename in os.listdir(label_dir):
    if filename.endswith('.json'):
        file_path = os.path.join(label_dir, filename)
        
        # JSON 파일 열기
        with open(file_path, 'r') as f:
            data = json.load(f)
            
            # JSON 데이터에서 신호등 객체 추출하여 색깔 확인
            objects = data.get('objects', [])
            for obj in objects:
                class_name = obj.get('class_name', None)
                attribute = obj.get('attribute', {})
                signal_color = attribute.get('signal', None)
                if signal_color:
                    signal_colors.add(signal_color)
                    if signal_color == 'etc':
                        # JSON 데이터에서 네 개의 꼭짓점 좌표 가져오기
                        points = obj.get('data', [])
                        if len(points) == 4:  # 네 개의 점이 주어진 경우
                            # 이미지 파일 경로
                            img_filename = os.path.splitext(filename)[0] + '.jpg'
                            img_path = os.path.join(image_dir, img_filename)
                            
                            # 이미지 불러오기
                            img = cv2.imread(img_path)
                            
                            # 네 개의 꼭짓점 좌표 추출
                            pt1 = (points[0][0], points[0][1])
                            pt2 = (points[1][0], points[1][1])
                            pt3 = (points[2][0], points[2][1])
                            pt4 = (points[3][0], points[3][1])
                            
                            # 네모 그리기
                            cv2.rectangle(img, pt1, pt3, (0, 0, 255), 2)
                            
                            # "etc" 텍스트 추가
                            cv2.putText(img, 'etc', pt1, cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                            
                            # 이미지 출력
                            cv2.imshow('Traffic Light', img)
                            cv2.waitKey(0)
                            cv2.destroyAllWindows()
print(signal_colors)

QFactoryLoader::QFactoryLoader() checking directory path "/home/min/venv/yolo/lib/python3.10/site-packages/cv2/qt/plugins" ...
QFactoryLoader::QFactoryLoader() checking directory path "/usr/bin" ...
QFactoryLoader::QFactoryLoader() looking at "/usr/bin/411toppm"
"Failed to extract plugin meta data from '/usr/bin/411toppm'" 
         not a plugin
QFactoryLoader::QFactoryLoader() looking at "/usr/bin/7z"
QElfParser: '/usr/bin/7z' is not an ELF object (file too small)
"'/usr/bin/7z' is not an ELF object (file too small)" 
         not a plugin
QFactoryLoader::QFactoryLoader() looking at "/usr/bin/7za"
QElfParser: '/usr/bin/7za' is not an ELF object (file too small)
"'/usr/bin/7za' is not an ELF object (file too small)" 
         not a plugin
QFactoryLoader::QFactoryLoader() looking at "/usr/bin/7zr"
QElfParser: '/usr/bin/7zr' is not an ELF object (file too small)
"'/usr/bin/7zr' is not an ELF object (file too small)" 
         not a plugin
QFactoryLoader::QFactoryLoader() looking at "/usr

{'red', 'red and green arrow', 'etc', 'green', 'None', 'yellow', 'green and green arrow', 'green arrow'}


넌 삭제다. etc

In [6]:
import os
import json

# JSON 파일이 있는 디렉토리 경로 설정
label_dir = '/home/min/dev_ws/machine_learning/project_3rd/data/traffic_detection/label'
# 처리된 JSON 파일을 저장할 디렉토리 경로 설정
output_dir = '/home/min/dev_ws/machine_learning/project_3rd/data/traffic_detection/processed_label'

# 모든 JSON 파일 탐색
for filename in os.listdir(label_dir):
    if filename.endswith('.json'):
        file_path = os.path.join(label_dir, filename)
        
        # JSON 파일 열기
        with open(file_path, 'r') as f:
            try:
                data = json.load(f)
            except json.JSONDecodeError:
                print(f"Error decoding JSON file: {file_path}")
                continue
            
            # JSON 데이터에서 "signal"이 "None"이거나 "etc"인 항목 제외하고 필터링
            if 'objects' in data:
                filtered_objects = [obj for obj in data['objects'] if obj.get('attribute', {}).get('signal') not in ['None', 'etc']]
                data['objects'] = filtered_objects
            else:
                print(f"Error: 'objects' key not found in JSON file: {file_path}")
                continue
            
        # 처리된 데이터를 새로운 JSON 파일로 저장
        output_path = os.path.join(output_dir, filename)
        with open(output_path, 'w') as f:
            json.dump(data, f, indent=4)



Error: 'objects' key not found in JSON file: /home/min/dev_ws/machine_learning/project_3rd/data/traffic_detection/label/512_ND_0006_CF_006.json
Error: 'objects' key not found in JSON file: /home/min/dev_ws/machine_learning/project_3rd/data/traffic_detection/label/512_ND_0011_CF_005.json
Error: 'objects' key not found in JSON file: /home/min/dev_ws/machine_learning/project_3rd/data/traffic_detection/label/512_ND_0002_CF_047.json
Error: 'objects' key not found in JSON file: /home/min/dev_ws/machine_learning/project_3rd/data/traffic_detection/label/512_ND_0007_CF_036.json
Error: 'objects' key not found in JSON file: /home/min/dev_ws/machine_learning/project_3rd/data/traffic_detection/label/512_ND_0004_CF_022.json


In [9]:
import os

# 디렉토리 구조 생성 함수
def create_directories(base_dir='/home/min/dev_ws/machine_learning/project_3rd/data/traffic_detection/YOLO_dataset'):
    dirs = [
        'images/train',
        'images/val',
        'images/test',
        'labels/train',
        'labels/val',
        'labels/test'
    ]
    for dir in dirs:
        os.makedirs(os.path.join(base_dir, dir), exist_ok=True)

create_directories()

yolo형식으로 바꾸기~~~~~!

In [7]:
import json
import os
import shutil
from sklearn.model_selection import train_test_split

# 경로 설정
image_dir = '/home/min/dev_ws/machine_learning/project_3rd/data/traffic_detection/image'
label_dir = '/home/min/dev_ws/machine_learning/project_3rd/data/traffic_detection/processed_label'

base_output_dir = '/home/min/dev_ws/machine_learning/project_3rd/data/traffic_detection/YOLO_dataset'
output_image_dir = os.path.join(base_output_dir, 'images')
output_label_dir = os.path.join(base_output_dir, 'labels')
# 출력 디렉터리 생성
def create_directories(base_dir='/home/min/dev_ws/machine_learning/project_3rd/data/traffic_detection/YOLO_dataset'):
    dirs = [
        'images/train',
        'images/val',
        'images/test',
        'labels/train',
        'labels/val',
        'labels/test'
    ]
    for dir in dirs:
        os.makedirs(os.path.join(base_dir, dir), exist_ok=True)

create_directories(base_output_dir)

# 클래스 딕셔너리
classes = {
    'red' : 0,
    'red and green arrow' :1 ,
    'green' : 2,
    'yellow' : 3,
    'green and green arrow' : 4,
    'green arrow' :5
}

# 바운딩 박스를 정규화하는 함수
def normalize_bbox(x_min, y_min, x_max, y_max, img_width, img_height):
    x_center = (x_min + x_max) / 2 / img_width
    y_center = (y_min + y_max) / 2 / img_height
    width = (x_max - x_min) / img_width
    height = (y_max - y_min) / img_height
    return x_center, y_center, width, height

# JSON 파일 목록을 가져옴
json_files = [f for f in os.listdir(label_dir) if f.endswith('.json')]

# 데이터를 train, val, test로 분할 (예: 70% train, 20% val, 10% test)
train_files, temp_files = train_test_split(json_files, test_size=0.3, random_state=42)
val_files, test_files = train_test_split(temp_files, test_size=1/3, random_state=42)

# 파일 복사 및 변환 함수
def process_files(file_list, image_output_dir, label_output_dir):
    for json_file in file_list:
        with open(os.path.join(label_dir, json_file), 'r', encoding='utf-8') as f:
            data = json.load(f)
            objects = data.get('objects', [])
        
        image_filename = data['image_name']
        img_width, img_height = data['image_size']
        
        # 이미지를 출력 디렉터리로 복사
        src_img_path = os.path.join(image_dir, image_filename)
        dst_img_path = os.path.join(image_output_dir, image_filename)
        if not os.path.exists(dst_img_path):
            shutil.copy(src_img_path, dst_img_path)
        
        # YOLO 형식 라벨 파일 생성
        label_filename = os.path.splitext(image_filename)[0] + '.txt'
        label_file_path = os.path.join(label_output_dir, label_filename)
        
        with open(label_file_path, 'w', encoding='utf-8') as label_file:
            for obj in objects:
                attribute = obj.get('attribute', {})
                signal = attribute.get('signal', None)
                
                # 클래스 ID 매핑
                if signal in classes:
                    class_id = classes[signal]
                else:
                    # 알 수 없는 신호일 경우 무시
                    continue
                
                points = obj.get('data', [])
                if len(points) == 4:
                    x_values = [point[0] for point in points]
                    y_values = [point[1] for point in points]
                    x_min = min(x_values)
                    x_max = max(x_values)
                    y_min = min(y_values)
                    y_max = max(y_values)
                    x_center, y_center, width, height = normalize_bbox(x_min, y_min, x_max, y_max, img_width, img_height)
                    label_file.write(f"{class_id} {x_center} {y_center} {width} {height}\n")
# 각 데이터셋에 대해 파일 처리
process_files(train_files, os.path.join(output_image_dir, 'train'), os.path.join(output_label_dir, 'train'))
process_files(val_files, os.path.join(output_image_dir, 'val'), os.path.join(output_label_dir, 'val'))
process_files(test_files, os.path.join(output_image_dir, 'test'), os.path.join(output_label_dir, 'test'))

In [8]:
import yaml

data = {
    'train': '/home/min/dev_ws/machine_learning/project_3rd/data/traffic_detection/YOLO_dataset/images/train',
    'val': '/home/min/dev_ws/machine_learning/project_3rd/data/traffic_detection/YOLO_dataset/images/val',
    'test': '/home/min/dev_ws/machine_learning/project_3rd/data/traffic_detection/YOLO_dataset/images/test',
    'nc': 6,
    'names': ['red',
              'red and green arrow',
              'green',
              'yellow',
              'green and green arrow',
              'green arrow']
}

with open('/home/min/dev_ws/machine_learning/project_3rd/data/traffic_detection/YOLO_dataset/dataset.yaml', 'w') as outfile:
    yaml.dump(data, outfile, default_flow_style=False)

In [9]:
from numba import cuda
device = cuda.get_current_device(); device.reset()

In [12]:
from ultralytics import YOLO

# 모델 초기화
model = YOLO('yolov5s.pt')
# 하이퍼파라미터 설정
train_params = {
    'data': '/home/min/dev_ws/machine_learning/project_3rd/data/traffic_detection/YOLO_dataset/dataset.yaml',
    'epochs': 20,
    'batch': 4,
    'imgsz': 640,
    'lr0': 0.001,          # 학습률을 낮춤
    'momentum': 0.8,       # 모멘텀 값을 낮춤
    'weight_decay': 0.0001, # 가중치 감쇠를 낮춤
    'optimizer': 'Adam',
    'patience': None 
}

# 훈련 중간 체크포인트 저장 경로 설정
checkpoint_path = 'yolo_checkpoint.pt'

# 모든 에폭을 돌면서 모델을 훈련
for epoch in range(train_params['epochs']):
    # 훈련 진행
    results = model.train(**train_params)
    
    # 중간 체크포인트 저장
    model.save(checkpoint_path)

# 훈련이 끝나면 최종 모델을 저장
final_model_path = 'final_yolo_model.pt'
model.save(final_model_path)

PRO TIP 💡 Replace 'model=yolov5s.pt' with new 'model=yolov5su.pt'.
YOLOv5 'u' models are trained with https://github.com/ultralytics/ultralytics and feature improved performance vs standard YOLOv5 models trained with https://github.com/ultralytics/yolov5.

New https://pypi.org/project/ultralytics/8.2.28 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.16 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 5938MiB)
engine/trainer: task=detect, mode=train, model=yolov5s.pt, data=/home/min/dev_ws/machine_learning/project_3rd/data/traffic_detection/YOLO_dataset/dataset.yaml, epochs=20, time=None, patience=None, batch=4, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train5, exist_ok=False, pretrained=True, optimizer=Adam, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, mult

train: Scanning /home/min/dev_ws/machine_learning/project_3rd/data/traffic_detection/YOLO_dataset/labels/train... 360 images, 5 backgrounds, 0 corrupt: 100%|██████████| 360/360 [00:00<00:00, 2562.35it/s]

train: New cache created: /home/min/dev_ws/machine_learning/project_3rd/data/traffic_detection/YOLO_dataset/labels/train.cache



val: Scanning /home/min/dev_ws/machine_learning/project_3rd/data/traffic_detection/YOLO_dataset/labels/val... 103 images, 1 backgrounds, 0 corrupt: 100%|██████████| 103/103 [00:00<00:00, 1083.10it/s]

val: New cache created: /home/min/dev_ws/machine_learning/project_3rd/data/traffic_detection/YOLO_dataset/labels/val.cache


...

       3/20      1.21G      1.584     0.9072     0.8227         13        640: 100%|██████████| 90/90 [00:09<00:00,  9.84it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 13/13 [00:01<00:00, 11.17it/s]

                   all        103        280      0.727      0.633      0.697      0.349



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       4/20      1.19G      1.671     0.9457     0.8183          8        640:  24%|██▍       | 22/90 [00:02<00:07,  8.91it/s]


KeyboardInterrupt: 